In [ ]:
#hide
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

%cd '/content/drive/MyDrive/spundsteif/spundsteif'
!pip3 install -e . -q
!pip3 install nbdev -q

#working directory: drive/MyDrive/spundsteif
%cd ..

%load_ext autoreload
%autoreload 2

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/spundsteif/spundsteif
ERROR: File "setup.py" or "setup.cfg" not found. Directory cannot be installed in editable mode: /content/drive/MyDrive/spundsteif/spundsteif
/content/drive/My Drive/spundsteif
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
# default_exp pressures

# Library pressures
> This library gives the earth pressure distribution along the sheet piles from the total stress given by the sensors and the piezometer data.  
> The computation of relative pressures compared to a given time is available.  
> Some functions aiming a graphical check of measurements are also available.  
--- 
**Inputs:**  
+ `df_messung`: dataframe of the measurements from sensors
+ `df_piezo.csv`: csv file of the groudwater level
+ `df_excavation.csv`: csv file of the stage of excavation

**Outputs:**  
+ `df_pressures` : total, effective stresses and waterpressure along the sheetpiles.


## Importation des bibliothèques

In [ ]:
#export
import pandas as pd
import numpy
import io
import csv
from datetime import datetime

from google.colab import files

from copy import copy

from datetime import timedelta
from datetime import datetime

import plotly.express as px
import plotly.graph_objects as go 

import numpy as np

from spundsteif import sensors

## Test setup

### Test files

In [ ]:
PIEZO_FILE_NAME_TEST = './test/df_piezo_test.csv'
EXCAVATION_FILE_NAME_TEST = './test/df_excavation_stage_test.csv'

# Files needed to get df_measured_pressure
STRUCTURE_DATA = './test/1_structure_file.txt'

TESTS_FILES_NAMES = ['./test/1_data_test_01.ASC',
                     './test/1_data_test_02.txt',
                     './test/1_data_test_03.txt',
                     './test/1_data_test_04.ASC'
                     ]

TEST_HAND = './test/df_hand_test.csv'

### Automatic measured pressure


In [ ]:
df_automatic_pressure = sensors.get_data(list_files_names=TESTS_FILES_NAMES,
                                         structure_data=STRUCTURE_DATA, 
                                         file_name_df_hand=TEST_HAND)

df_automatic_pressure = df_automatic_pressure.loc[:,df_automatic_pressure.columns.str.startswith('EDS')]

In [ ]:
df_automatic_pressure.sample(10, random_state=0)

,EDS_A,EDS_B,EDS_C,EDS_D,EDS_E,EDS_F,EDS_G,EDS_B2
2021-07-12,0.002243,5.255263,0.089889,NaN,0.007241,-6.078723,5.736801,NaN
2021-07-09,-0.002018,3.480844,-0.044311,NaN,0.012253,-6.144872,0.760613,NaN
2021-07-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-18,0.006446,NaN,2.545558,NaN,-0.001639,-6.176094,-3.770186,0.849900
2021-08-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-08-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-07-21,-0.009055,NaN,2.535880,NaN,-0.031098,-6.303607,-4.200203,4.607471


In [ ]:
df_automatic_pressure.shape

(25, 8)

In [ ]:
df2 = df_automatic_pressure.dropna()
df2.shape

(0, 8)

## Read files

### Read raw data

In [ ]:
#export
def construct_df_csv(file_name):
  df = pd.read_csv(file_name, delimiter=';')

  #remove empty lines and columns
  df = df.dropna(how='all')
  df = df.dropna(how='all', axis='columns')
  
  return df

In [ ]:
# example
df_piezo = construct_df_csv(PIEZO_FILE_NAME_TEST)
df_piezo

,Datum,Zeit,Tiefe/Top Spundwand [m],Temp [°C]
0,06.07.2021,10:21:00,2.20,14.4
1,07.07.2021,12:00:00,2.15,17.3
2,14.07.2021,10:30:00,1.92,14.5
3,21.07.2021,09:30:00,3.33,13.1
4,28.07.2021,10:00:00,2.11,12.3
5,05.08.2021,09:00:00,2.02,13.0
6,11.08.2021,09:00:00,2.43,14.8
7,18.08.2021,09:30:00,2.40,18.0
8,25.08.2021,11:10:00,2.62,13.6


In [ ]:
# example
df_excavation = construct_df_csv(EXCAVATION_FILE_NAME_TEST)
df_excavation

,Datum,Zeit,Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
0,07.07.2021,07:30:00,1.1,1.1,1.0
1,21.07.2021,07:30:00,1.1,1.1,0.5
2,28.07.2021,08:30:00,1.8,1.1,0.5
3,18.08.2021,09:30:00,1.8,1.1,0.5
4,25.08.2021,10:30:00,1.8,1.1,0.5


### Clean data

In [ ]:
#export
def clean(df):
  
  # dtypes
  df['date'] = pd.to_datetime(df['Datum'], format='%d.%m.%Y')

  # remove 'Zeit' and 'Datum'
  df = df.drop(columns=['Datum', 'Zeit'])

  df = df.set_index('date')

  return df

In [ ]:
# example
df_piezo = clean(df_piezo)
df_piezo.dtypes

Tiefe/Top Spundwand [m]    float64
Temp [°C]                  float64
dtype: object

In [ ]:
# example
df_excavation= clean(df_excavation)
df_excavation.dtypes

Aushubstiefe/Verklebtebohlen [m]    float64
Aushubstiefe/Einzelbohlen [m]       float64
Tiefe/Aktivenseite [m]              float64
dtype: object

In [ ]:
# test
for df in df_piezo, df_excavation:
  assert pd.api.types.is_datetime64_any_dtype(df.index)
  assert 'Datum' not in df.columns
  assert 'Zeit' not in df.columns

assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Verklebtebohlen [m]'])
assert pd.api.types.is_float_dtype(df_excavation['Aushubstiefe/Einzelbohlen [m]'])

assert pd.api.types.is_float_dtype(df_piezo['Tiefe/Top Spundwand [m]'])
assert pd.api.types.is_float_dtype(df_piezo['Temp [°C]'])

## Merge data

### Manual measures

All measures are depth in [m] from the top of the sheet piles

#### Merge piezo and excavation

In [ ]:
#export
def merge_piezo_excavation(df_piezo, df_excavation):
  df = df_piezo.join(df_excavation, how='outer')
  return df

In [ ]:
# example
df_manual = merge_piezo_excavation(df_piezo, df_excavation)
df_manual

,Tiefe/Top Spundwand [m],Temp [°C],Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m]
date,,,,,
2021-07-06,2.20,14.4,NaN,NaN,NaN
2021-07-07,2.15,17.3,1.1,1.1,1.0
2021-07-14,1.92,14.5,NaN,NaN,NaN
2021-07-21,3.33,13.1,1.1,1.1,0.5
2021-07-28,2.11,12.3,1.8,1.1,0.5
2021-08-05,2.02,13.0,NaN,NaN,NaN
2021-08-11,2.43,14.8,NaN,NaN,NaN
2021-08-18,2.40,18.0,1.8,1.1,0.5
2021-08-25,2.62,13.6,1.8,1.1,0.5


#### Clean manual measures

In [ ]:
#export
def clean_manual_df(df):
  df['GWT [m]'] = df['Tiefe/Top Spundwand [m]']
  df = df.drop(columns=['Tiefe/Top Spundwand [m]'])
  return df

In [ ]:
# example
df_manual = clean_manual_df(df_manual)
df_manual

,Temp [°C],Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,NaN,NaN,NaN,2.20
2021-07-07,17.3,1.1,1.1,1.0,2.15
2021-07-14,14.5,NaN,NaN,NaN,1.92
2021-07-21,13.1,1.1,1.1,0.5,3.33
2021-07-28,12.3,1.8,1.1,0.5,2.11
2021-08-05,13.0,NaN,NaN,NaN,2.02
2021-08-11,14.8,NaN,NaN,NaN,2.43
2021-08-18,18.0,1.8,1.1,0.5,2.40
2021-08-25,13.6,1.8,1.1,0.5,2.62


In [ ]:
# test
assert df_manual['GWT [m]']['2021-07-06'] == 2.2

#### Upsample to 1 day

In [ ]:
def upsample_manual_df(df):
  df = df.asfreq(freq='d')
  return df

In [ ]:
# example
e = upsample_manual_df(df_manual)
e

,Temp [°C],Aushubstiefe/Verklebtebohlen [m],Aushubstiefe/Einzelbohlen [m],Tiefe/Aktivenseite [m],GWT [m]
date,,,,,
2021-07-06,14.4,NaN,NaN,NaN,2.20
2021-07-07,17.3,1.1,1.1,1.0,2.15
2021-07-08,NaN,NaN,NaN,NaN,NaN
2021-07-09,NaN,NaN,NaN,NaN,NaN
2021-07-10,NaN,NaN,NaN,NaN,NaN
2021-07-11,NaN,NaN,NaN,NaN,NaN
2021-07-12,NaN,NaN,NaN,NaN,NaN
2021-07-13,NaN,NaN,NaN,NaN,NaN
2021-07-14,14.5,NaN,NaN,NaN,1.92


In [ ]:
# test
#TODO

#### Interpolate

In [ ]:
def interpolate_manual_df(df):
  df = df.interpolate(method='time')
  df = df.fillna(method='bfill')
  # TODO: for each colums add a "intepolated column" boolean
  return df

In [ ]:
# example
# TODO

In [ ]:
# test
assert e['Aushubstiefe/Verklebtebohlen [m]']['2021-07-06'] == 1.1
assert e['Aushubstiefe/Einzelbohlen [m]']['2021-07-06'] == 1.1

#TODO add tests

AssertionError: ignored

#### Get manual dataframe

In [ ]:
#export
def get_manual_data(piezo_file_name, excavation_file_name):
  df_piezo = construct_df_csv(piezo_file_name)
  df_piezo = clean(df_piezo)

  df_excavation = construct_df_csv(excavation_file_name)
  df_excavation = clean(df_excavation)

  df = merge_piezo_excavation(df_piezo, df_excavation)
  df = clean_manual_df(df)
  df = upsample_manual_df(df)
  df = interpolate_manual_df(df)

  return df

In [ ]:
# example
df_manual = get_manual_data(PIEZO_FILE_NAME_TEST, EXCAVATION_FILE_NAME_TEST)

fig = go.Figure()
cols = df_manual.columns.drop('Temp [°C]')

for col in cols:
  fig.add_trace(
      go.Scatter(x=df_manual.index, y=df_manual[col], 
                  name=col, 
                  mode='lines+markers', 
                  marker_size=3
      )
  )

# Mise en page
fig.update_layout(
    margin=dict(l=30, r=30, t=50, b=30),
    title='Manual data',
    xaxis_title="Date",
    yaxis_title="Depth in m",
    title_font=dict(family="Rockwell", size=18),
    yaxis = dict(autorange="reversed")
)

fig.show()

In [ ]:
# test
# TODO

### Merge manual and automatic measures

#### Merging

In [ ]:
#export
def merge_manual_automatic(df_manual, df_automatic):
    df = df_automatic.join(df_manual, how='outer')
    return df

#### Interpolation

In [ ]:
# needed?

In [ ]:
# example
from plotly.subplots import make_subplots
fig = make_subplots(specs=[[{"secondary_y": True}]])

e = merge_manual_automatic(df_manual, df_automatic_pressure)
e = interpolate(e)
e = resample(e)
cols = e.columns.drop('Temp [°C]')

for col in cols:
    fig.add_trace(
        go.Scatter(x=e.index,
                  y=e[col], 
                  name=col, 
                  mode='lines+markers',
                  marker_size=3 if 'EDS' in col else 5),
        secondary_y=False if 'EDS' in col else True,
    )

# Mise en page
fig.update_layout(
    margin=dict(l=30, r=30, t=50, b=30),
    title='Manual data',
    xaxis_title="Date",
    title_font=dict(family="Rockwell", size=18),
)

   
fig.update_yaxes(title_text="Depth in m", secondary_y=False)
fig.update_yaxes(title_text="Presure in MPa", secondary_y=True)

fig.show()

In [ ]:
#hide
from nbdev.export import notebook2script; notebook2script()